# Prompt Chaining for Complex Workflows

## Overview

**Prompt chaining** breaks complex tasks into sequential LLM calls, passing outputs as inputs to subsequent steps. This pattern is essential for:

1. **Reliability** - Smaller focused tasks vs one complex prompt
2. **Modularity** - Test and debug each step independently
3. **Flexibility** - Mix models, temperatures, and parameters per step
4. **State Management** - Track progress through multi-step workflows

## Why Chaining Beats Single Prompts

### Single Complex Prompt Issues
- ❌ Lower accuracy (too many instructions)
- ❌ Harder to debug (black box)
- ❌ All-or-nothing failure
- ❌ Can't optimize per-step

### Chained Approach Benefits
- ✅ Higher accuracy (focused tasks)
- ✅ Clear error isolation
- ✅ Granular error handling
- ✅ Cost optimization (cheap models for simple steps)

## This Notebook's Chain

**3-Step Customer Service Pipeline:**

```
Step 1: Extract Intent → Parse user query for products/categories
         ↓
Step 2: Fetch Data → Retrieve product info from catalog
         ↓
Step 3: Generate Response → Answer with context
```

## Key Concepts

### State Passing
Output of Step N becomes input to Step N+1

### Error Handling
Each step can fail gracefully without breaking the chain

### Context Injection
Intermediate data enriches final LLM call

## Environment Setup

In [ ]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

---

## Step 1: Extract Products and Categories

**Goal**: Parse user query to identify mentioned products/categories

**System Message Strategy**:
- Comprehensive product catalog in prompt (6 categories, 30+ products)
- JSON output for programmatic parsing
- Handle both specific products ("SmartX ProPhone") and categories ("TVs")

**Output Format**:
```python
[
    {"products": ["SmartX ProPhone", "FotoSnap DSLR Camera"]},
    {"category": "Televisions and Home Theater Systems"}
]
```

**Why This Works**:
- Structured output enables Step 2 (data fetching)
- Explicit product list prevents hallucination
- Category grouping reduces lookups

In [ ]:
client = openai.OpenAI()

def get_completion_from_messages(
    messages,
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=500,
):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content


**Expected Output**:
```python
[
    {"products": ["SmartX ProPhone", "FotoSnap DSLR Camera"]},
    {"category": "Televisions and Home Theater Systems"}
]
```

---

## Step 2: Product Catalog (Data Source)

In production, this would be a database or API. For this demo, it's a Python dict with 30+ products across 6 categories.

**Product Schema**:
```python
{
    "name": str,
    "category": str,
    "brand": str,
    "model_number": str,
    "warranty": str,
    "rating": float,
    "features": list[str],
    "description": str,
    "price": float
}
```

In [ ]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Output a python list of objects, where each object has \
the following format:
    'category': <one of Computers and Laptops, \
    Smartphones and Accessories, \
    Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, 
    Audio Equipment, Cameras and Camcorders>,
OR
    'products': <a list of products that must \
    be found in the allowed products below>

Where the categories and products must be found in \
the customer service query.
If a product is mentioned, it must be associated with \
the correct category in the allowed products list below.
If no products or categories are found, output an \
empty list.

Allowed products: 

Computers and Laptops category:
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

Smartphones and Accessories category:
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

Televisions and Home Theater Systems category:
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

Gaming Consoles and Accessories category:
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

Audio Equipment category:
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

Cameras and Camcorders category:
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

Only output the list of objects, with nothing else.
"""
user_message_1 = f"""
 tell me about the smartx pro phone and \
 the fotosnap camera, the dslr one. \
 Also tell me about your tvs """
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

## Helper Functions: Data Retrieval

These functions bridge Step 1 (LLM extraction) and Step 3 (LLM response generation).

In [ ]:
# product information
products = {
    "TechPro Ultrabook": {
        "name": "TechPro Ultrabook",
        "category": "Computers and Laptops",
        "brand": "TechPro",
        "model_number": "TP-UB100",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["13.3-inch display", "8GB RAM", "256GB SSD", "Intel Core i5 processor"],
        "description": "A sleek and lightweight ultrabook for everyday use.",
        "price": 799.99
    },
    "BlueWave Gaming Laptop": {
        "name": "BlueWave Gaming Laptop",
        "category": "Computers and Laptops",
        "brand": "BlueWave",
        "model_number": "BW-GL200",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["15.6-inch display", "16GB RAM", "512GB SSD", "NVIDIA GeForce RTX 3060"],
        "description": "A high-performance gaming laptop for an immersive experience.",
        "price": 1199.99
    },
    "PowerLite Convertible": {
        "name": "PowerLite Convertible",
        "category": "Computers and Laptops",
        "brand": "PowerLite",
        "model_number": "PL-CV300",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["14-inch touchscreen", "8GB RAM", "256GB SSD", "360-degree hinge"],
        "description": "A versatile convertible laptop with a responsive touchscreen.",
        "price": 699.99
    },
    "TechPro Desktop": {
        "name": "TechPro Desktop",
        "category": "Computers and Laptops",
        "brand": "TechPro",
        "model_number": "TP-DT500",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["Intel Core i7 processor", "16GB RAM", "1TB HDD", "NVIDIA GeForce GTX 1660"],
        "description": "A powerful desktop computer for work and play.",
        "price": 999.99
    },
    "BlueWave Chromebook": {
        "name": "BlueWave Chromebook",
        "category": "Computers and Laptops",
        "brand": "BlueWave",
        "model_number": "BW-CB100",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["11.6-inch display", "4GB RAM", "32GB eMMC", "Chrome OS"],
        "description": "A compact and affordable Chromebook for everyday tasks.",
        "price": 249.99
    },
    "SmartX ProPhone": {
        "name": "SmartX ProPhone",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-PP10",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["6.1-inch display", "128GB storage", "12MP dual camera", "5G"],
        "description": "A powerful smartphone with advanced camera features.",
        "price": 899.99
    },
    "MobiTech PowerCase": {
        "name": "MobiTech PowerCase",
        "category": "Smartphones and Accessories",
        "brand": "MobiTech",
        "model_number": "MT-PC20",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["5000mAh battery", "Wireless charging", "Compatible with SmartX ProPhone"],
        "description": "A protective case with built-in battery for extended usage.",
        "price": 59.99
    },
    "SmartX MiniPhone": {
        "name": "SmartX MiniPhone",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-MP5",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["4.7-inch display", "64GB storage", "8MP camera", "4G"],
        "description": "A compact and affordable smartphone for basic tasks.",
        "price": 399.99
    },
    "MobiTech Wireless Charger": {
        "name": "MobiTech Wireless Charger",
        "category": "Smartphones and Accessories",
        "brand": "MobiTech",
        "model_number": "MT-WC10",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["10W fast charging", "Qi-compatible", "LED indicator", "Compact design"],
        "description": "A convenient wireless charger for a clutter-free workspace.",
        "price": 29.99
    },
    "SmartX EarBuds": {
        "name": "SmartX EarBuds",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-EB20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "24-hour battery life"],
        "description": "Experience true wireless freedom with these comfortable earbuds.",
        "price": 99.99
    },

    "CineView 4K TV": {
        "name": "CineView 4K TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-4K55",
        "warranty": "2 years",
        "rating": 4.8,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "A stunning 4K TV with vibrant colors and smart features.",
        "price": 599.99
    },
    "SoundMax Home Theater": {
        "name": "SoundMax Home Theater",
        "category": "Televisions and Home Theater Systems",
        "brand": "SoundMax",
        "model_number": "SM-HT100",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth"],
        "description": "A powerful home theater system for an immersive audio experience.",
        "price": 399.99
    },
    "CineView 8K TV": {
        "name": "CineView 8K TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-8K65",
        "warranty": "2 years",
        "rating": 4.9,
        "features": ["65-inch display", "8K resolution", "HDR", "Smart TV"],
        "description": "Experience the future of television with this stunning 8K TV.",
        "price": 2999.99
    },
    "SoundMax Soundbar": {
        "name": "SoundMax Soundbar",
        "category": "Televisions and Home Theater Systems",
        "brand": "SoundMax",
        "model_number": "SM-SB50",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth"],
        "description": "Upgrade your TV's audio with this sleek and powerful soundbar.",
        "price": 199.99
    },
    "CineView OLED TV": {
        "name": "CineView OLED TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-OLED55",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "Experience true blacks and vibrant colors with this OLED TV.",
        "price": 1499.99
    },

    "GameSphere X": {
        "name": "GameSphere X",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-X",
        "warranty": "1 year",
        "rating": 4.9,
        "features": ["4K gaming", "1TB storage", "Backward compatibility", "Online multiplayer"],
        "description": "A next-generation gaming console for the ultimate gaming experience.",
        "price": 499.99
    },
    "ProGamer Controller": {
        "name": "ProGamer Controller",
        "category": "Gaming Consoles and Accessories",
        "brand": "ProGamer",
        "model_number": "PG-C100",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["Ergonomic design", "Customizable buttons", "Wireless", "Rechargeable battery"],
        "description": "A high-quality gaming controller for precision and comfort.",
        "price": 59.99
    },
    "GameSphere Y": {
        "name": "GameSphere Y",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-Y",
        "warranty": "1 year",
        "rating": 4.8,
        "features": ["4K gaming", "500GB storage", "Backward compatibility", "Online multiplayer"],
        "description": "A compact gaming console with powerful performance.",
        "price": 399.99
    },
    "ProGamer Racing Wheel": {
        "name": "ProGamer Racing Wheel",
        "category": "Gaming Consoles and Accessories",
        "brand": "ProGamer",
        "model_number": "PG-RW200",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Force feedback", "Adjustable pedals", "Paddle shifters", "Compatible with GameSphere X"],
        "description": "Enhance your racing games with this realistic racing wheel.",
        "price": 249.99
    },
    "GameSphere VR Headset": {
        "name": "GameSphere VR Headset",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-VR",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["Immersive VR experience", "Built-in headphones", "Adjustable headband", "Compatible with GameSphere X"],
        "description": "Step into the world of virtual reality with this comfortable VR headset.",
        "price": 299.99
    },

    "AudioPhonic Noise-Canceling Headphones": {
        "name": "AudioPhonic Noise-Canceling Headphones",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-NC100",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["Active noise-canceling", "Bluetooth", "20-hour battery life", "Comfortable fit"],
        "description": "Experience immersive sound with these noise-canceling headphones.",
        "price": 199.99
    },
    "WaveSound Bluetooth Speaker": {
        "name": "WaveSound Bluetooth Speaker",
        "category": "Audio Equipment",
        "brand": "WaveSound",
        "model_number": "WS-BS50",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Portable", "10-hour battery life", "Water-resistant", "Built-in microphone"],
        "description": "A compact and versatile Bluetooth speaker for music on the go.",
        "price": 49.99
    },
    "AudioPhonic True Wireless Earbuds": {
        "name": "AudioPhonic True Wireless Earbuds",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-TW20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "18-hour battery life"],
        "description": "Enjoy music without wires with these comfortable true wireless earbuds.",
        "price": 79.99
    },
    "WaveSound Soundbar": {
        "name": "WaveSound Soundbar",
        "category": "Audio Equipment",
        "brand": "WaveSound",
        "model_number": "WS-SB40",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["2.0 channel", "80W output", "Bluetooth", "Wall-mountable"],
        "description": "Upgrade your TV's audio with this slim and powerful soundbar.",
        "price": 99.99
    },
    "AudioPhonic Turntable": {
        "name": "AudioPhonic Turntable",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-TT10",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["3-speed", "Built-in speakers", "Bluetooth", "USB recording"],
        "description": "Rediscover your vinyl collection with this modern turntable.",
        "price": 149.99
    },

    "FotoSnap DSLR Camera": {
        "name": "FotoSnap DSLR Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-DSLR200",
        "warranty": "1 year",
        "rating": 4.7,
        "features": ["24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses"],
        "description": "Capture stunning photos and videos with this versatile DSLR camera.",
        "price": 599.99
    },
    "ActionCam 4K": {
        "name": "ActionCam 4K",
        "category": "Cameras and Camcorders",
        "brand": "ActionCam",
        "model_number": "AC-4K",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["4K video", "Waterproof", "Image stabilization", "Wi-Fi"],
        "description": "Record your adventures with this rugged and compact 4K action camera.",
        "price": 299.99
    },
    "FotoSnap Mirrorless Camera": {
        "name": "FotoSnap Mirrorless Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-ML100",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["20.1MP sensor", "4K video", "3-inch touchscreen", "Interchangeable lenses"],
        "description": "A compact and lightweight mirrorless camera with advanced features.",
        "price": 799.99
    },
    "ZoomMaster Camcorder": {
        "name": "ZoomMaster Camcorder",
        "category": "Cameras and Camcorders",
        "brand": "ZoomMaster",
        "model_number": "ZM-CM50",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["1080p video", "30x optical zoom", "3-inch LCD", "Image stabilization"],
        "description": "Capture life's moments with this easy-to-use camcorder.",
        "price": 249.99
    },
    "FotoSnap Instant Camera": {
        "name": "FotoSnap Instant Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-IC10",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["Instant prints", "Built-in flash", "Selfie mirror", "Battery-powered"],
        "description": "Create instant memories with this fun and portable instant camera.",
        "price": 69.99
    }
}

---

## Step 2a: Parse JSON Response

Convert LLM's text response (JSON string) into Python data structure.

In [ ]:
def get_product_by_name(name):
    return products.get(name, None)

def get_products_by_category(category):
    return [product for product in products.values() if product["category"] == category]

## Step 2b: Fetch Product Data

Use parsed list to retrieve full product details from catalog.

In [ ]:
import json 

def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace("'", "\"")  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None   
    

## Execute Step 2: Parse and Fetch

Convert Step 1's output into actionable data structure.

In [ ]:
def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string 

## Fetch Full Product Details

Use parsed list to retrieve complete product information.

In [ ]:
category_and_product_list = read_string_to_list(category_and_product_response_1)
print(category_and_product_list)

**Result**: Complete product details (specs, pricing, features) for:
- SmartX ProPhone
- FotoSnap DSLR Camera
- All TVs in catalog (5 products)

---

## Step 3: Generate Contextual Response

**Goal**: Answer user's question with fetched product data as context

**Key Pattern**: Inject product data as **assistant message** (not system message)

### Why Assistant Message?
- **System**: Sets overall behavior
- **Assistant**: Provides context/knowledge for current query
- Simulates "the assistant already researched this"

**Message Structure**:
```python
[
    {"role": "system", "content": "Be helpful and concise"},
    {"role": "user", "content": "Tell me about X and Y"},
    {"role": "assistant", "content": "Relevant product info: {...}"},  # Context injection
]
```

The LLM treats assistant's "research" as context when crafting its response.

In [ ]:
product_information_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_information_for_user_message_1)

**Expected Response**: Concise product descriptions with follow-up questions

Example:
```
The SmartX ProPhone features a 6.1-inch display, 128GB storage, and 5G for $899.99. 
The FotoSnap DSLR Camera has a 24.2MP sensor and 1080p video for $599.99.

We have 5 great TV options from $599.99 to $2999.99, including 4K, 8K, and OLED models.

What features are most important to you? Are you interested in a specific screen size or smart TV capabilities?
```

---

## Complete Chain Visualization

```
┌─────────────────────────────────────────┐
│ User: "Tell me about SmartX ProPhone,  │
│        FotoSnap DSLR, and your TVs"     │
└──────────────┬──────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│ STEP 1: Extract Intent (LLM)            │
│ Output: [{"products": [...]},           │
│          {"category": "TVs"}]           │
└──────────────┬──────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│ STEP 2a: Parse JSON (Python)            │
│ Output: Python list of dicts            │
└──────────────┬──────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│ STEP 2b: Fetch Data (Python)            │
│ Output: Full product details (JSON)     │
└──────────────┬──────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│ STEP 3: Generate Response (LLM)         │
│ Input: User query + Product context     │
│ Output: Friendly, concise answer        │
└──────────────┬──────────────────────────┘
               │
               ▼
        User sees response
```

---

## Production Implementation Patterns

### Error Handling at Each Step

```python
def process_user_query(user_message):
    try:
        # Step 1: Extract intent
        extraction_response = extract_products_and_categories(user_message)
        if not extraction_response:
            return "I'm having trouble understanding. Could you rephrase?"
        
        # Step 2a: Parse JSON
        parsed_data = read_string_to_list(extraction_response)
        if not parsed_data:
            log_error("JSON parsing failed", extraction_response)
            return "Sorry, I'm experiencing technical difficulties."
        
        # Step 2b: Fetch data
        product_data = generate_output_string(parsed_data)
        if not product_data:
            return "I couldn't find information on those products."
        
        # Step 3: Generate response
        final_response = generate_user_response(user_message, product_data)
        return final_response
        
    except Exception as e:
        log_error("Chain failed", str(e))
        return "I apologize, something went wrong. Please try again."
```

### Cost Optimization

**Model Selection Per Step**:
```python
# Step 1: Simple extraction (cheap model)
extraction = get_completion(
    messages, 
    model="gpt-3.5-turbo",  # $0.50/1M input tokens
    temperature=0
)

# Step 3: User-facing response (quality matters)
response = get_completion(
    messages,
    model="gpt-4o",  # $2.50/1M input tokens, better quality
    temperature=0.7  # Slightly creative for friendliness
)
```

**Cost Analysis**:
- Step 1: ~200 tokens (extraction) = $0.0001
- Step 2: 0 tokens (Python only)
- Step 3: ~500 tokens (response) = $0.0013 (gpt-4o)
- **Total per query**: ~$0.0014

### Caching Strategy

```python
import hashlib
import redis

cache = redis.Redis()

def get_product_data_cached(parsed_list):
    # Generate cache key from product list
    cache_key = hashlib.md5(str(parsed_list).encode()).hexdigest()
    
    # Check cache
    cached_data = cache.get(cache_key)
    if cached_data:
        return cached_data.decode()
    
    # Fetch from catalog
    data = generate_output_string(parsed_list)
    
    # Cache for 1 hour
    cache.setex(cache_key, 3600, data)
    return data
```

### Async Processing for Speed

```python
import asyncio

async def process_chain_async(user_message):
    # Step 1: Extract (LLM call)
    extraction = await extract_intent_async(user_message)
    
    # Step 2a: Parse (fast, no await needed)
    parsed = read_string_to_list(extraction)
    
    # Step 2b: Fetch from DB (async)
    product_data = await fetch_products_async(parsed)
    
    # Step 3: Generate response (LLM call)
    response = await generate_response_async(user_message, product_data)
    
    return response
```

---

## Advanced Chaining Patterns

### 1. Conditional Branching

```python
def process_with_branches(user_message):
    # Step 1: Classify intent
    intent = classify_intent(user_message)
    
    if intent == "product_inquiry":
        # Branch A: Product info chain
        return product_info_chain(user_message)
    elif intent == "support":
        # Branch B: Support ticket chain
        return support_chain(user_message)
    elif intent == "complaint":
        # Branch C: Escalation chain
        return escalation_chain(user_message)
    else:
        return general_response(user_message)
```

### 2. Iterative Refinement

```python
def iterative_chain(user_message):
    # Step 1: Generate draft
    draft = generate_draft(user_message)
    
    # Step 2: Self-critique
    critique = evaluate_response(draft)
    
    # Step 3: Refine based on critique
    if critique["score"] < 0.7:
        refined = refine_response(draft, critique["issues"])
        return refined
    
    return draft
```

### 3. Parallel Chains

```python
async def parallel_chains(user_message):
    # Run multiple chains concurrently
    results = await asyncio.gather(
        extract_products(user_message),
        check_inventory(user_message),
        fetch_user_history(user_id)
    )
    
    products, inventory, history = results
    
    # Combine results in final step
    return generate_personalized_response(products, inventory, history)
```

---

## Monitoring and Debugging

### Chain Telemetry

```python
def process_with_telemetry(user_message):
    trace_id = generate_trace_id()
    
    # Step 1
    start = time.time()
    extraction = extract_intent(user_message)
    log_step(trace_id, "extract", time.time() - start, len(extraction))
    
    # Step 2
    start = time.time()
    data = fetch_products(extraction)
    log_step(trace_id, "fetch", time.time() - start, len(data))
    
    # Step 3
    start = time.time()
    response = generate_response(user_message, data)
    log_step(trace_id, "generate", time.time() - start, len(response))
    
    return response
```

### Failure Analysis

Track where chains break:
```python
{
    "trace_id": "abc123",
    "step_1_success": True,
    "step_1_latency_ms": 450,
    "step_2_success": False,  # Failed here
    "step_2_error": "JSON parse error",
    "step_3_executed": False
}
```

---

## Summary: Chaining Best Practices

### When to Use Chaining
✅ Multi-step workflows (classify → fetch → respond)  
✅ Need for intermediate data (database lookups)  
✅ Error isolation required (fail gracefully per step)  
✅ Different models/temperatures per step  
✅ Complex business logic between LLM calls

### When NOT to Chain
❌ Simple single-turn Q&A  
❌ No intermediate processing needed  
❌ Latency is critical (<500ms target)  
❌ Added complexity not justified

### Key Takeaways
1. **Reliability**: Smaller tasks = higher accuracy
2. **Debuggability**: Isolate failures to specific steps
3. **Flexibility**: Mix LLM and traditional code
4. **Cost**: Optimize model selection per step
5. **Monitoring**: Track each step's performance

### Next Steps
- **F.EvaluatingOutputs** - Assess chain quality
- **G.CustomerServiceBot** - Full system with chaining
- **H.EvaluatingLLMPerformance** - Benchmark chains vs single prompts

In [ ]:
system_message = f"""
You are a customer service assistant for a \
large electronic store. \
Respond in a friendly and helpful tone, \
with very concise answers. \
Make sure to ask the user relevant follow up questions.
"""
user_message_1 = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs"""
messages =  [  
{'role':'system',
 'content': system_message},   
{'role':'user',
 'content': user_message_1},  
{'role':'assistant',
 'content': f"""Relevant product information:\n\
 {product_information_for_user_message_1}"""},   
]
final_response = get_completion_from_messages(messages)
print(final_response)